In [ ]:
from google.colab import drive

In [ ]:
drive.flush_and_unmount()

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 1. Face Detection


In [ ]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import os

In [ ]:
import albumentations as alb
from PIL import Image

1.1 Haar-like

In [ ]:
face_cascade = cv2.CascadeClassifier( './haarcascade_frontalface_default.xml' )
eye_cascade = cv2.CascadeClassifier( './haarcascade_eye.xml' )

In [ ]:
def get_cropped_image_if_2_eyes ( image_path ) :
  img = cv2.imread ( image_path )
  gray = cv2.cvtColor ( img , cv2.COLOR_BGR2GRAY )
  faces = face_cascade.detectMultiScale ( gray , 1.3 , 5 )
  for ( x , y , w , h ) in faces :
    roi_gray =  gray [ y : y + h , x : x + w ]
    roi_color = img [ y : y + h , x : x + w ]
    eyes = eye_cascade.detectMultiScale ( roi_gray )
    if len ( eyes ) >= 2:
      return roi_color

In [ ]:
for dir in os.walk('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_data'):
  if os.path.basename(os.path.normpath(dir[0])) == 'final_data':
        continue
  os.makedirs('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_reco_data/'+ os.path.basename(os.path.normpath(dir[0])))
  print(os.path.basename(os.path.normpath(dir[0])))
  count = 0
  for image in os.listdir(dir[0]):
    face = get_cropped_image_if_2_eyes('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_data/'+os.path.basename(os.path.normpath(dir[0]))+'/'+image)
    if face is not None:
      print(count)
      count = count + 1
      plt.imshow(face)
      cv2.imwrite ( './gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_reco_data/'+ os.path.basename(os.path.normpath(dir[0]))+'/'+image , face )


1.2 Custom neural network for further filtering

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np

In [ ]:
facetracker = load_model('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/facetracker.h5')

In [ ]:
for dir in os.walk('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/data'):
  count = 0
  for image in os.listdir(dir[0]):
    if os.path.basename(os.path.normpath(dir[0])) == 'data':
        continue
    if count == 0:
      os.makedirs('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/reco_data/'+ os.path.basename(os.path.normpath(dir[0])))
    count = 1
    print(image)
    img = cv2.imread('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/data/'+ os.path.basename(os.path.normpath(dir[0])) + '/' + image)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    if yhat[0] > 0.9:
      img = Image.open( './gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/data/'+  os.path.basename(os.path.normpath(dir[0])) +'/' + image )
      img.save('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/reco_data/'+ os.path.basename(os.path.normpath(dir[0])) + '/' + image)

1.3 Image augmentation

In [ ]:
def augmentor_creator():
  augmentor = alb.Compose([alb.HorizontalFlip(p=0.5),
                          alb.RandomBrightnessContrast (p=0.4),
                          alb.RandomGamma (p=0.4),
                          alb.RGBShift(p=0.2),
                          alb.Blur(blur_limit=3, p=0.3),
                          alb.HueSaturationValue(p=0.3),
                          alb.ShiftScaleRotate(rotate_limit=10, p=0.3)]
                          )
  return augmentor

In [ ]:
for dir in os.walk('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_aug_data/'): 
  count = 0
  if os.path.basename(os.path.normpath(dir[0])) != 'Didier_Drogba':
      continue
  #os.makedirs("./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_aug_data/"+os.path.basename(os.path.normpath(dir[0])))
  print(os.path.basename(os.path.normpath(dir[0])))
  for image in os.listdir(dir[0]): 
    img = cv2.imread ( './gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_aug_data/'+os.path.basename(os.path.normpath(dir[0]))+'/'+image, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor ( img , cv2.COLOR_BGR2GRAY )
    img = cv2.imread(os.path.join(dir[0],image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(img, (300,300), interpolation = cv2.INTER_AREA)
    for x in range(2):
      count = count + 1
      print(count)
      data = {'image':resized}
      augmentor = augmentor_creator()
      augmented = augmentor(**data)
      im = Image.fromarray(augmented['image'])
      im.save("./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_aug_data/"+os.path.basename(os.path.normpath(dir[0]))+"/"+ f'{image.split(".")[0]}.{x}.jpg')
    

# Feature extraction using LBP

In [ ]:
import shutil
shutil.rmtree('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/aug_data', ignore_errors=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import imageio
from skimage.feature import local_binary_pattern
import os
import cv2
import tensorflow as tf
from PIL import Image
from skimage.transform import integral_image

In [ ]:
class LocalBinaryPatterns:
  def __init__(self , numPoints , radius):
    self.numPoints = numPoints
    self.radius = radius

  def calculate_region(self, image, k, eps=1e-7):
    store_all = []
    count_region = 0
    image = np.asarray(image)
    window_size_row = image.shape[0] / k
    window_size_column = image.shape[1] / k
    window_size_row = int(window_size_row)
    window_size_column = int(window_size_column)
    for i in range(0, image.shape[0] - window_size_row, window_size_row):
        for r in range(0, image.shape[1] - window_size_column, window_size_column):
          count_region = count_region + 1
          region = image[i:i + window_size_row, r:r + window_size_column]
          LBP = local_binary_pattern(region, self.numPoints, self.radius, method="uniform")
          (hist, _) = np.histogram(LBP, bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))
          hist = hist.astype("float")
          hist /= (hist.sum()+eps)
          store_all.extend(hist)
    return store_all

  def describe(self , image , eps=1e-7):
      hist = self.calculate_region(image, 7)
      return hist

In [ ]:
desc = LocalBinaryPatterns(8 , 2)

In [ ]:
features = []
labels = []
temp = []
lcount = 0
class_names = []
for dir in os.walk('./gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_aug_data/'): 
  if os.path.basename(os.path.normpath(dir[0])) == 'final_aug_data':
      continue
  class_names.append(os.path.basename(os.path.normpath(dir[0])))
  print(os.path.basename(os.path.normpath(dir[0])))
  #if lcount == 3:
  #  break
  count = 0
  #os.mkdir("./lbp/"+os.path.basename(os.path.normpath(dir[0])))
  for image in os.listdir(dir[0]):
    img = cv2.imread ( './gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/final_aug_data/' + os.path.basename(os.path.normpath(dir[0])) + '/' + image )
    resized = cv2.resize(img, (300,300))
    gray = cv2.cvtColor (resized , cv2.COLOR_BGR2GRAY )
    hist=desc.describe(gray)
    # save lbp as an image for tensorflow deep learning model
    #im = Image.fromarray(lbp)
    #if im.mode != 'RGB':
    #  im = im.convert('RGB')
    #im.save("./lbp/"+os.path.basename(os.path.normpath(dir[0]))+"/"+image)
    #save lbp as an array for svm
    temp.append(hist)
    labels.append(lcount)
    print(count)
    count = count + 1
  lcount = lcount + 1
features = np.array(temp)
labels = np.array(labels)

Didier_Drogba
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273


In [ ]:
class_names

['Didier_Drogba', 'Lionel_Messi', 'Zlatan_Ibrahimovic', 'Cristiano_Ronaldo']

# Classification

1.   SVM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,labels,train_size=0.9, test_size=0.1)

In [ ]:
model = SVC(gamma='auto', probability=True, C=1000, kernel='rbf')                         
model.fit(x_train, y_train)

SVC(C=1000, gamma='auto', probability=True)

In [ ]:
model.score(x_test, y_test)

0.8126984126984127

In [ ]:
model.predict([hist])

array([0])

In [ ]:
import pickle
pkl_filename = "final_SVC_facerecognizer.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

2. MLP

In [ ]:
import tensorflow as tf
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices ('GPU')
for gpu in gpus :
  tf.config.experimental.set_memory_growth (gpu, True) 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os
import cv2
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(features,labels,train_size=0.8, test_size=0.2,random_state=1)

In [ ]:
x_test,x_val,y_test,y_val = train_test_split(x_test,y_test,train_size=0.5, test_size=0.5,random_state=1)

In [ ]:
model = Sequential([
  layers.Dense(4000, activation='relu', input_dim=490),
  layers.Dense(2000, activation='relu'),
  layers.Dense(300, activation='relu'),
  layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=40
history = model.fit(
  x_train,y_train, 
  validation_data=(x_val,y_val),
  epochs=epochs
)

Epoch 1/40
56/56 [==============================] - 7s 113ms/step - loss: 1.0584 - accuracy: 0.4330 - val_loss: 0.9884 - val_accuracy: 0.4821
Epoch 2/40
56/56 [==============================] - 5s 96ms/step - loss: 0.9704 - accuracy: 0.5190 - val_loss: 0.9558 - val_accuracy: 0.5312
Epoch 3/40
56/56 [==============================] - 5s 90ms/step - loss: 0.8692 - accuracy: 0.6011 - val_loss: 0.7831 - val_accuracy: 0.6696
Epoch 4/40
56/56 [==============================] - 5s 92ms/step - loss: 0.7313 - accuracy: 0.6944 - val_loss: 0.7978 - val_accuracy: 0.6473
Epoch 5/40
56/56 [==============================] - 5s 90ms/step - loss: 0.6633 - accuracy: 0.7123 - val_loss: 0.7822 - val_accuracy: 0.6473
Epoch 6/40
56/56 [==============================] - 5s 91ms/step - loss: 0.6448 - accuracy: 0.7251 - val_loss: 0.7565 - val_accuracy: 0.6652
Epoch 7/40
56/56 [==============================] - 5s 91ms/step - loss: 0.5724 - accuracy: 0.7598 - val_loss: 0.6105 - val_accuracy: 0.7321
Epoch 8/40
5

In [ ]:
model.evaluate(x_test,y_test)

7/7 [==============================] - 0s 19ms/step - loss: 0.5943 - accuracy: 0.8125


[0.5943294763565063, 0.8125]

In [ ]:
len(x_val[0])

490

In [ ]:
model.save('./MLP_face_recognizer.h5')

In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/personal archive/WISD_Courses/S2/Visual Analytics/Project/face_recognizer.h5')

In [ ]:

img = cv2.imread ('./didier drogba png308.jpg')
resized = cv2.resize(img, (300,300))
gray = cv2.cvtColor (resized , cv2.COLOR_BGR2GRAY )
hist=desc.describe(gray)



In [ ]:

img_array = tf.expand_dims(hist, 0) # Create a batch

predictions = model.predict(img_array)
score = predictions[0]

In [ ]:
np.around(predictions[0])

array([0., 1., 0.], dtype=float32)